In [27]:
from collections import Counter
from datetime import datetime
import json

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from IntegratedGradients import * 
from keras.models import load_model

import numpy as np
# caleb_path = '/Users/CalebKaijiLu/Documents/yelp_dataset/'
caleb_path = '/home/caleb/schoolwork/yelp_dataset/'
selva_path = ''

In [29]:
def word_to_embedding(embeddings,data):
    N = embeddings.shape[0]
    M = embeddings.shape[1]
    embeddings_aug = np.zeros([N+1,M])
    embeddings_aug[1:,:]=embeddings
    return embeddings_aug[np.array(data)]

In [2]:

t1 = datetime.now()
with open(caleb_path + "review.json") as f:
    reviews = f.read().strip().split("\n")
# reviews = reviews[:10000]
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

0:01:01.728785


In [13]:
# Get a balanced sample of positive and negative reviews
# texts = [review['text'] for review in reviews]
reviews = [review for review in reviews if review['stars']!=3 ]
# Convert our 5 classes into 2 (negative or positive)
binstars = [0 if review['stars'] < 3 else 1 for review in reviews]
balanced_texts = []
balanced_labels = []
limit = 200000  #Change this to grow/shrink the dataset
neg_pos_counts = [0, 0]
for i in range(len(texts)):
    polarity = binstars[i]
    if neg_pos_counts[polarity]<limit:
        balanced_texts.append(texts[i])
        balanced_labels.append(binstars[i])
        neg_pos_counts[polarity]+=1

In [14]:
Counter(balanced_labels)

Counter({0: 200000, 1: 200000})

In [15]:
tokenizer = Tokenizer(num_words=5)
toytexts = ["Is is a common word", "So is the", "the is common", "discombobulation is not common"]
tokenizer.fit_on_texts(toytexts)
sequences = tokenizer.texts_to_sequences(toytexts)


# >>> [[1, 1, 4, 2], [1, 3], [3, 1, 2], [1, 2]]

In [16]:
print(sequences)
# >>> [[1, 1, 4, 2], [1, 3], [3, 1, 2], [1, 2]]

[[1, 1, 4, 2], [1, 3], [3, 1, 2], [1, 2]]


In [17]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(balanced_texts)
sequences = tokenizer.texts_to_sequences(balanced_texts)
data = pad_sequences(sequences, maxlen=300)

In [19]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data, np.array(balanced_labels), validation_split=0.5, epochs=3,batch_size = 32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 128)          2560000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 300, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 296, 64)           41024     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 74, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,699,969
Trainable params: 2,699,969
Non-trainable params: 0
_________________________________________________________________


In [21]:
# save the tokenizer and model
import pickle
with open("keras_tokenizer_server.pickle", "wb") as f:
        pickle.dump(tokenizer, f)
model.save("yelp_sentiment_model_server.hdf5")

In [25]:
with open("keras_tokenizer_server.pickle", "rb") as f:
       tokenizer = pickle.load(f)

model = load_model("yelp_sentiment_model_server.hdf5")

In [30]:
# replace with the data you want to classify
newtexts = ["Really hate the tacos, they tastes awful"]
word_sequences = text_to_word_sequence(newtexts[0])
# note that we shouldn't call "fit" on the tokenizer again
sequences = tokenizer.texts_to_sequences(newtexts)
data = pad_sequences(sequences, maxlen=300)

# get predictions for each of your new texts
predictions = model.predict(data)
print(predictions)
print(word_sequences)

embeddings = model.layers[0].get_weights()[0]
word_embeddings = word_to_embedding(embeddings,data)
ig = integrated_gradients(model)
explanations = ig.explain(word_to_embedding(embeddings,data))
print(len(explanations))
final_explanations = np.abs(np.sum(explanations[0],axis = 1))
print(np.where(final_explanations!=0)[0])
final_explanations = final_explanations[np.where(final_explanations!=0)]
print(final_explanations.shape)
y_pos = np.arange(len(final_explanations))
plt.bar(y_pos,final_explanations,align='center', alpha=0.5)
plt.xticks(y_pos, word_sequences)
plt.ylabel('Integrated Gradients')
plt.title('Explanations for word in Review')
plt.show()

[[ 0.01438663]]
['really', 'hate', 'the', 'tacos', 'they', 'tastes', 'awful']
(2,)
Hahahahahahah
Evaluated output channel (0-based index): All
Building gradient functions


TypeError: Can not convert a NoneType into a Tensor or Operation.